<a href="https://colab.research.google.com/github/alfcastillo90/ApuntesBigData/blob/main/Tarea_1_bigdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Tarea
# 1) Seleccionar un producto
# 2) Identificar las sesiones de carrito que tiene ese producto
# 3) identificar los productos que pertenecen al mismo carrito que el producto seleccionado


In [2]:
#Hacemos un montaje en google drive
from google.colab import drive

drive.mount('/content/gdrive')

gdrive = 'content/gdrive/MyDrive/Estudios/UBO/BIGDATA'

Mounted at /content/gdrive


In [3]:
#Instalamos todas las librerias necesarias
%%capture
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget 'https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz' -O 'spark-3.3.1-bin-hadoop3.tgz'
!tar -xvf /content/spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark

In [4]:
#Definimos las variables de entorno
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME'] = '/content/spark-3.3.1-bin-hadoop3'

In [5]:
#Construimos la sesion de spark
import findspark
findspark.init()

from pyspark.sql import SparkSession

#El asterisco representa que se vana  ocupar todos los recursos
spark = SparkSession.builder.master("local[*]").appName('tarea1').getOrCreate()


In [7]:
#descomprimimos el archivo
!unzip '/content/gdrive/MyDrive/Estudios/UBO/BIGDATA/archive.zip'

Archive:  /content/gdrive/MyDrive/Estudios/UBO/BIGDATA/archive.zip
  inflating: 2019-Dec.csv            
  inflating: 2019-Nov.csv            
  inflating: 2019-Oct.csv            
  inflating: 2020-Feb.csv            
  inflating: 2020-Jan.csv            


In [8]:
#Creamos el data frame
df = spark.read.options(header='True', inferSchema='True').csv('/content/*.csv')


In [13]:
#Creamos una tabla temporal
df.createOrReplaceTempView('data');

In [16]:
#Identificamos los campos
df.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [15]:

#listamos los productos cuyo tipo de evento sea igual a carrito, con el fin de escoger el que tenga el contador mayor
spark.sql('select product_id, count(*) contador from data where event_type="cart" group by product_id order by contador desc limit 1').show()

+----------+--------+
|product_id|contador|
+----------+--------+
|   5700037|   28347|
+----------+--------+



In [21]:
#product_id escogido
productId = "5700037"
print('product_id escogido', productId)
#Identificamos las sesiones del carrito que tienen ese producto
userSessionsQuery = 'select user_session from data where event_type="cart" and product_id="'+productId+'" limit 10'
spark.sql(userSessionsQuery).show()

product_id escogido 5700037
+--------------------+
|        user_session|
+--------------------+
|9caff1dd-7437-456...|
|112e47bb-3af7-419...|
|d545859b-eb3f-473...|
|d545859b-eb3f-473...|
|d545859b-eb3f-473...|
|bb6ece2f-f127-633...|
|bb6ece2f-f127-633...|
|bb6ece2f-f127-633...|
|bb6ece2f-f127-633...|
|809cc5d7-55db-450...|
+--------------------+



In [23]:
#Identificamos los otros productos que comparten carrito con el id de producto seleccionado previamente
productUserSessionQuery = 'select product_id from data where user_session in ('+userSessionsQuery+')'
spark.sql(productUserSessionQuery).show()

+----------+
|product_id|
+----------+
|   5862564|
|   5587652|
|   5587658|
|   5896429|
|   5896421|
|   5896421|
|   5587652|
|   5587652|
|   5896429|
|   5896429|
|   5896429|
|   5700037|
|   5700038|
|   5565816|
|   5700049|
|   5565816|
|   5565816|
|   5565816|
|   5695956|
|   5758630|
+----------+
only showing top 20 rows

